In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import pickle

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from gensim.models import Word2Vec, KeyedVectors

In [42]:
df = pd.read_csv("C:/Users/Malathi M/OneDrive/Documents/MDTE25/guvi final project/Main project/news.tsv.zip", sep="\t")
print(df.head())

  News ID Category         Topic  \
0  N10000   sports        soccer   
1  N10001     news  newspolitics   
2  N10002     news        newsus   
3  N10003     news  newspolitics   
4  N10004     news     newsworld   

                                            Headline  \
0  Predicting Atlanta United's lineup against Col...   
1  Mitch McConnell: DC statehood push is 'full bo...   
2            Home In North Highlands Damaged By Fire   
3  Meghan McCain blames 'liberal media' and 'thir...   
4                            Today in History: Aug 1   

                                           News body  \
0  Only FIVE internationals allowed, count em, FI...   
1  WASHINGTON -- Senate Majority Leader Mitch McC...   
2  NORTH HIGHLANDS (CBS13)   Fire damaged a home ...   
3  Meghan McCain is speaking out after a journali...   
4  1714: George I becomes King Georg Ludwig, Elec...   

                                Title entity  \
0  {"Atlanta United's": 'Atlanta United FC'}   
1            

In [43]:
# Fill NaN values to avoid errors
df['Headline'] = df.get('Headline', '').fillna('').astype(str)
df['News body'] = df.get('News body', '').fillna('').astype(str)

In [44]:
df.isnull().sum()

News ID           0
Category          0
Topic             0
Headline          0
News body         0
Title entity      0
Entity content    0
dtype: int64

In [45]:
# Combine Headline + News body
df['text'] = (df['Headline'].str.strip() + ' ' + df['News body'].str.strip()).str.strip()
df['text'] = df['text'].fillna('')

# Feature and target
X = df["text"]
y = df["Category"]

In [50]:
pip install transformers datasets torch scikit-learn accelerate


In [56]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["label"] = le.fit_transform(df["Category"])


In [57]:
num_classes = len(le.classes_)
print("Number of classes:", num_classes)


Number of classes: 16


In [58]:
print(df[["Category", "label"]].head())
print(df["label"].value_counts())


         Category  label
99542        news     10
111370         tv     13
21981   lifestyle      7
24460      health      5
84028      sports     11
label
11    7582
10    6723
3     2655
7     1817
4     1404
1     1394
12    1338
14    1198
13    1011
5      938
15     819
9      634
8      467
2      388
6       71
0        1
Name: count, dtype: int64


In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Use only 50% of data (as you mentioned)
df = df.sample(frac=0.5, random_state=42)

train_df, temp_df = train_test_split(
    df, test_size=0.2, stratify=df["label"], random_state=42
)

val_df, test_df = train_test_split(
    temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42
)


In [60]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))


distilbert base predefine

In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "distilbert-base-uncased"   
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


In [ ]:
def tokenize(batch):
    return tokenizer(
        batch["text"],      
        padding="max_length",
        truncation=True,
        max_length=128
    )


In [18]:
train_ds = train_ds.map(tokenize, batched=True)
val_ds   = val_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)


Map:   0%|          | 0/5688 [00:00<?, ? examples/s]

Map:   0%|          | 0/711 [00:00<?, ? examples/s]

Map:   0%|          | 0/711 [00:00<?, ? examples/s]

In [19]:
train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [20]:
from transformers import AutoModelForSequenceClassification

num_labels = df["label"].nunique()

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

c:\Users\Malathi M\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Malathi M\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-

In [22]:
import transformers
print(transformers.__version__)


4.57.3


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",        
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)


In [24]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted"
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


C:\Users\Malathi M\AppData\Local\Temp\ipykernel_5384\1482414042.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\Malathi M\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.335800,0.865846,0.742616,0.737807,0.742616,0.733019
2,0.805600,0.802949,0.752461,0.755807,0.752461,0.749858
3,0.535400,0.805558,0.752461,0.755842,0.752461,0.751043


c:\Users\Malathi M\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Malathi M\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Malathi M\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Malathi M\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is se

TrainOutput(global_step=2133, training_loss=0.8130622377319622, metrics={'train_runtime': 5392.4708, 'train_samples_per_second': 3.164, 'train_steps_per_second': 0.396, 'total_flos': 565236934871040.0, 'train_loss': 0.8130622377319622, 'epoch': 3.0})

In [26]:
test_results = trainer.evaluate(test_ds)
print(test_results)


c:\Users\Malathi M\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.7261126637458801, 'eval_accuracy': 0.7932489451476793, 'eval_precision': 0.7882601771999407, 'eval_recall': 0.7932489451476793, 'eval_f1': 0.7896491274691464, 'eval_runtime': 51.983, 'eval_samples_per_second': 13.678, 'eval_steps_per_second': 1.712, 'epoch': 3.0}


c:\Users\Malathi M\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
trainer.save_model("./news_classifier_transformer")
tokenizer.save_pretrained("./news_classifier_transformer")


Bert pertained

In [75]:
import torch
import numpy as np
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, f1_score


In [77]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding=True,
        truncation=True,
        max_length=100
    )


In [78]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
val_ds   = Dataset.from_pandas(val_df)
test_ds  = Dataset.from_pandas(test_df)

train_ds = train_ds.map(tokenize, batched=True)
val_ds   = val_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)

cols = ["input_ids", "attention_mask", "label"]
train_ds.set_format("torch", columns=cols)
val_ds.set_format("torch", columns=cols)
test_ds.set_format("torch", columns=cols)


Map:   0%|          | 0/11376 [00:00<?, ? examples/s]

Map:   0%|          | 0/1422 [00:00<?, ? examples/s]

Map:   0%|          | 0/1422 [00:00<?, ? examples/s]

In [79]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_classes
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#def compute_metrics(pred):
 #   labels = pred.label_ids
  #  preds = np.argmax(pred.predictions, axis=1)
   # return {
    #    "accuracy": accuracy_score(labels, preds),
     #   "f1": f1_score(labels, preds, average="weighted")
    #}


In [ ]:
#pip install --upgrade transformers


In [81]:
training_args = TrainingArguments(
    output_dir="./bert_results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs"
)


In [ ]:
#from transformers import Trainer, TrainingArguments

#from sklearn.metrics import accuracy_score, f1_score

#def compute_metrics(p):
 #   preds = p.predictions.argmax(-1)
  #  labels = p.label_ids
  #  acc = accuracy_score(labels, preds)
  #  f1 = f1_score(labels, preds, average="weighted")
  #  return {"accuracy": acc, "f1": f1}

#trainer = Trainer(
 #   model=model,
  #  args=training_args,
  #  train_dataset=train_ds,
  #  eval_dataset=val_ds,
  #  compute_metrics=compute_metrics
#)


In [82]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted"
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [83]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics
)

In [84]:
trainer.train()


Step,Training Loss
500,1.361600
1000,0.940700


TrainOutput(global_step=1422, training_loss=1.0733407150676315, metrics={'train_runtime': 8684.0057, 'train_samples_per_second': 1.31, 'train_steps_per_second': 0.164, 'total_flos': 584673360537600.0, 'train_loss': 1.0733407150676315, 'epoch': 1.0})

In [85]:
trainer.save_model("./bert_classification_model")
tokenizer.save_pretrained("./bert_classification_model")

('./bert_classification_model\\tokenizer_config.json',
 './bert_classification_model\\special_tokens_map.json',
 './bert_classification_model\\vocab.txt',
 './bert_classification_model\\added_tokens.json',
 './bert_classification_model\\tokenizer.json')